# Gozumol - Colab Test Notebook

Bu notebook, Gozumol kütüphanesini Google Colab'da GPU ile test etmek için hazırlanmıştır.

**Adımlar:**
1. GPU'yu etkinleştir (Runtime > Change runtime type > GPU)
2. Hücreleri sırayla çalıştır

---

## 1. GPU Kontrolü

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU Aktif: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("GPU BULUNAMADI! Runtime > Change runtime type > GPU seçin!")

## 2. Bağımlılıkları Kur

**ÖNEMLİ:** Phi-4 modeli belirli versiyonlar gerektirir. Kurulumdan sonra **Runtime > Restart session** yapın!

In [ ]:
# Phi-4 için uyumlu versiyonları kur
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124
!pip install transformers==4.48.2
!pip install accelerate==1.3.0
!pip install pillow requests

print("\n" + "="*60)
print("Bağımlılıklar kuruldu!")
print("="*60)
print("\n⚠️ ÖNEMLİ: Şimdi Runtime > Restart session yapın!")
print("Sonra Adım 3'ten devam edin (bu hücreyi tekrar çalıştırmayın)")

## 3. Gozumol Kurulumu

**⚠️ Restart yaptıktan sonra buradan devam edin!**

In [ ]:
# Versiyon kontrolü
import torch
import transformers
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

# GitHub'dan repo'yu klonla
!rm -rf /content/gozumol 2>/dev/null
!git clone https://github.com/eminaruk/gozumol.git
%cd /content/gozumol

# Gozumol'u yükle
!pip install -q -e .

print("\nKurulum tamamlandı!")

## 4. Import Testi

In [ ]:
from gozumol import VisionAssistant
from gozumol.prompts import get_prompt_pair
from gozumol.core.utils import get_device_info

# Device bilgisi
info = get_device_info()
print(f"Device: {info['device']}")
if info.get('cuda_device_name'):
    print(f"GPU: {info['cuda_device_name']}")
    print(f"VRAM: {info['cuda_memory_total_gb']:.1f} GB")

print("\nImport başarılı!")

## 5. Model Yükleme

**Not:** İlk çalıştırmada model indirilecek (~11GB). Bu birkaç dakika sürebilir.

In [ ]:
# VisionAssistant'ı oluştur (model yüklenecek)
print("Model yükleniyor... (ilk seferde ~5-10 dakika sürebilir)")

assistant = VisionAssistant(
    scenario='outdoor',  # Dış mekan senaryosu
    device='cuda'        # GPU kullan
)

print(f"\n{assistant}")
print("Model hazır!")

## 6. Test Görselleri Yükle

In [ ]:
import requests
from PIL import Image
from IPython.display import display

# Test görselleri
test_images = {
    "sokak_1": "https://heytripster.com/wp-content/uploads/2023/08/Trastevere-rome.jpg",
    "sokak_2": "https://oitheblog.com/wp-content/uploads/2016/03/SAM_7237.jpg",
}

images = {}
for name, url in test_images.items():
    try:
        img = Image.open(requests.get(url, stream=True, timeout=10).raw)
        images[name] = img
        print(f"{name}: {img.size} - Yüklendi")
    except Exception as e:
        print(f"{name}: Yüklenemedi - {e}")

print(f"\n{len(images)} görsel yüklendi!")

In [ ]:
# Görselleri göster
for name, img in images.items():
    print(f"\n--- {name} ---")
    display(img.resize((400, 300)))

## 7. Test: Temel Çevre Açıklaması

In [ ]:
# İlk görsel ile test
print("Görsel analiz ediliyor...\n")

response, timing = assistant.describe(images["sokak_1"], return_timing=True)

print("=" * 60)
print("ÇEVRE AÇIKLAMASI")
print("=" * 60)
print(response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 8. Test: Hızlı Güvenlik Taraması

In [ ]:
print("Hızlı güvenlik taraması...\n")

quick_response, timing = assistant.quick_scan(images["sokak_2"], return_timing=True)

print("=" * 60)
print("HIZLI GÜVENLİK TARAMASI")
print("=" * 60)
print(quick_response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 9. Test: Karşıdan Karşıya Geçiş

In [ ]:
print("Karşıdan karşıya geçiş kontrolü...\n")

crossing_response, timing = assistant.check_crossing(images["sokak_1"], return_timing=True)

print("=" * 60)
print("GEÇİŞ GÜVENLİĞİ")
print("=" * 60)
print(crossing_response)
print("=" * 60)
print(f"\nSüre: {timing['total_time']:.2f}s")

## 10. Kendi Görselini Test Et

In [ ]:
from google.colab import files

print("Görsel yüklemek için aşağıdaki butona tıkla:")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    custom_image = Image.open(filename)
    print(f"\nYüklenen: {filename} ({custom_image.size})")
    display(custom_image.resize((400, 300)))

In [ ]:
# Yüklenen görseli test et
if 'custom_image' in dir():
    assistant.set_scenario('outdoor')
    
    print("Görsel analiz ediliyor...\n")
    response, timing = assistant.describe(custom_image, return_timing=True)
    
    print("=" * 60)
    print("ÇEVRE AÇIKLAMASI")
    print("=" * 60)
    print(response)
    print("=" * 60)
    print(f"\nSüre: {timing['total_time']:.2f}s")
else:
    print("Önce yukarıdaki hücrede bir görsel yükle!")

## 11. Performans Özeti

In [ ]:
import torch

# GPU bellek kullanımı
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated() / 1024**3
    reserved = torch.cuda.memory_reserved() / 1024**3
    total = torch.cuda.get_device_properties(0).total_memory / 1024**3
    
    print("GPU Bellek Kullanımı:")
    print(f"  - Ayrılmış: {allocated:.2f} GB")
    print(f"  - Rezerve: {reserved:.2f} GB")
    print(f"  - Toplam: {total:.2f} GB")
    print(f"  - Kullanım: {(reserved/total)*100:.1f}%")

print("\nTest Tamamlandı!")